# Tarea 8: Modelo de urnas

En este experimento se tiene una cantidad ```k``` de cúmulos y una cantidad ```n```de partículas, cumpliendo la condición de que si se suman las partículas contenidas en cada uno de los ```k```cúmulos esto da exactamente ```n```.

Se trabaja con la función ```modelournas```, la cual está basada en el código de Schaeffer (https://github.com/satuelisa/Simulation/blob/master/UrnModel/aggrFrag.R), que depende de los parámetros ```k```, ```n```y ```duracion``` (la cuál es la cantidad de pasos que realiza la simulación), dicha función calcula el porcentaje de cúmulos que tienen una cantidad de partículas mayores a ```c```, donde este valor es la media de los cúmulos formados al inicio del proceso.

In [52]:
modelournas <- function(k, n, duracion){
    originales <- rnorm(k)
    cumulos <- originales - min(originales) + 1
    cumulos <- round(n * cumulos / sum(cumulos))
    assert(min(cumulos) > 0)
    diferencia <- n - sum(cumulos)
    if (diferencia > 0) {
        for (i in 1:diferencia) {
            p <- sample(1:k, 1)
            cumulos[p] <- cumulos[p] + 1
        }
    } else if (diferencia < 0) {
        for (i in 1:-diferencia) {
            p <- sample(1:k, 1)
            if (cumulos[p] > 1) {
                cumulos[p] <- cumulos[p] - 1
            }
        }
    }
    assert(length(cumulos[cumulos == 0]) == 0) # que no haya vacios
    assert(sum(cumulos) == n)
    c <- median(cumulos) # tamanio critico de cumulos
    d <- sd(cumulos) / 4 # factor arbitrario para suavizar la curva
    rotura <- function(x) {
        return (1 / (1 + exp((c - x) / d)))
    }
    union <- function(x) {
        return (exp(-x / c))
    }
    romperse <- function(tam, cuantos) {
        romper <- round(rotura(tam) * cuantos) # independientes
        resultado <- rep(tam, cuantos - romper) # los demas
        if (romper > 0) {
            for (cumulo in 1:romper) { # agregar las rotas
                t <- 1
                if (tam > 2) { # sample no jala con un solo valor
                    t <- sample(1:(tam-1), 1)
                }
                resultado <- c(resultado, t, tam - t)
            }
        }
        assert(sum(resultado) == tam * cuantos) # no hubo perdidas
        return(resultado)
    }
    unirse <- function(tam, cuantos) {
        unir <- round(union(tam) * cuantos) # independientes
        if (unir > 0) {
            division <- c(rep(-tam, unir), rep(tam, cuantos - unir))
            assert(sum(abs(division)) == tam * cuantos)
            return(division)
        } else {
            return(rep(tam, cuantos))
        }
    }
    freq <- as.data.frame(table(cumulos))
    names(freq) <- c("tam", "num")
    freq$tam <- as.numeric(levels(freq$tam))[freq$tam]

    for (paso in 1:duracion) {
        assert(sum(cumulos) == n)
        cumulos <- integer()
        for (i in 1:dim(freq)[1]) { # fase de rotura
            urna <- freq[i,]
            if (urna$tam > 1) { # no tiene caso romper si no se puede
                cumulos <- c(cumulos, romperse(urna$tam, urna$num))
            } else {
                cumulos <- c(cumulos, rep(1, urna$num))
            }
        }
        assert(sum(cumulos) == n)
        assert(length(cumulos[cumulos == 0]) == 0) # que no haya vacios
        freq <- as.data.frame(table(cumulos)) # actualizar urnas
        names(freq) <- c("tam", "num")
        freq$tam <- as.numeric(levels(freq$tam))[freq$tam]
        assert(sum(freq$num * freq$tam) == n)
        cumulos <- integer()
        for (i in 1:dim(freq)[1]) { # fase de union
            urna <- freq[i,]
            cumulos <- c(cumulos, unirse(urna$tam, urna$num))
        }
        assert(sum(abs(cumulos)) == n)
        assert(length(cumulos[cumulos == 0]) == 0) # que no haya vacios
        juntarse <- -cumulos[cumulos < 0]
        cumulos <- cumulos[cumulos > 0]
        assert(sum(cumulos) + sum(juntarse) == n)
        nt <- length(juntarse)
        if (nt > 0) {
            if (nt > 1) {
                juntarse <- sample(juntarse)
                for (i in 1:floor(nt / 2) ) {
                    cumulos <- c(cumulos, juntarse[2*i-1] + juntarse[2*i])
                }
            }
            if (nt %% 2 == 1) {
                cumulos <- c(cumulos, juntarse[nt])
            }
        }
        assert(sum(cumulos) == n)
        freq <- as.data.frame(table(cumulos))
        names(freq) <- c("tam", "num")
        freq$tam <- as.numeric(levels(freq$tam))[freq$tam]
        assert(sum(freq$num * freq$tam) == n)
        cum <- c()

        for (i in 1:length(freq$num)){
            if(freq$tam[i]>=c){
                cum <- c(cum, freq$num[i])
            }
        }
        a<-sum(cum)/sum(freq$num)

    }
    return(a)
}

Para la tarea base, se pide que se varíen los valores de los parámetros para determinar el porcentaje de los cúmulos que sí se pueden filtrar. El parámetro ```k```varía en $\{100, 1000, 10000\}$, ```n``` en $\{10k, 20k, 50k, 100k, 1000k\}$ y ```duracion```en $\{10, 25, 50, 75, 100\}$, cada una las posibles combinaciones se replica una cantidad de 25 veces. 

In [278]:
data <- data.frame()
cums <- c(100, 1000, 10000, 100000, 1000000)
parts <- c(10*cums, 20*cums, 50*cums, 100*cums, 1000*cums)
dur <- c(10, 25, 50, 75,100)
replicas <- 25

for (k in cums){
    for (n in parts){
        for (duracion in dur){
            for (j in 1:replicas){
                    por <- modelournas(k,n,duracion)
                    data <- rbind(data, c(k,n,duracion, j,por))
            }
        }
    }
}




Con ```k=100```

In [489]:
png("100.png", width = 900, height = 500)
boxplot(datos$porcentaje[1:25], datos$porcentaje[26:50], datos$porcentaje[51:75], datos$porcentaje[76:100], datos$porcentaje[101:125],
        datos$porcentaje[126:150], datos$porcentaje[151:175], datos$porcentaje[176:200], datos$porcentaje[201:225], datos$porcentaje[226:250],
        datos$porcentaje[251:275], datos$porcentaje[276:300], datos$porcentaje[301:325], datos$porcentaje[326:350], datos$porcentaje[351:400],
        datos$porcentaje[401:425], datos$porcentaje[426:450], datos$porcentaje[451:475], datos$porcentaje[476:500], 
        ylab="Porcentaje de cúmulos filtrados", col=c("pink3","pink3","pink3","pink3","pink3","plum3","plum3","plum3","plum3","plum3",
                                                     "lightskyblue3","lightskyblue3", "lightskyblue3", "lightskyblue3", "lightskyblue3",
                                                     "mediumaquamarine", "mediumaquamarine", "mediumaquamarine", "mediumaquamarine", "mediumaquamarine",
                                                     "orange2","orange2","orange2","orange2","orange2"), xlab="Duración", names=c(10, 25, 50, 75,100,
                                                                                                                                 10, 25, 50, 75,100,
                                                                                                                                 10, 25, 50, 75,100,
                                                                                                                                 10, 25, 50, 75),
        ylim=c(0.28,0.45))
legend(x = "topright", legend = c("10k", "20k", "50k","100k", "1000k"), fill = c("pink3", "plum3", "lightskyblue3", "mediumaquamarine", "orange2"), 
       title = "Partículas")
dev.off()

pdf 
  2

Con ```k=1000```

In [488]:
png("1000.png", width = 900, height = 500)
boxplot(datos_1$porcentaje[1:25], datos_1$porcentaje[26:50], datos_1$porcentaje[51:75], datos_1$porcentaje[76:100], datos_1$porcentaje[101:125],
        datos_1$porcentaje[126:150], datos_1$porcentaje[151:175], datos_1$porcentaje[176:200], datos_1$porcentaje[201:225], datos_1$porcentaje[226:250],
        datos_1$porcentaje[251:275], datos_1$porcentaje[276:300], datos_1$porcentaje[301:325], datos_1$porcentaje[326:350], datos_1$porcentaje[351:375],
        datos_1$porcentaje[376:400], datos_1$porcentaje[401:425], datos_1$porcentaje[426:450], datos_1$porcentaje[451:475], datos_1$porcentaje[476:500], 
        datos_1$porcentaje[501:525], datos_1$porcentaje[526:550], datos_1$porcentaje[551:575], datos_1$porcentaje[576:600], datos_1$porcentaje[600:625],
        ylab="Porcentaje de cúmulos filtrados", col=c("pink3","pink3","pink3","pink3","pink3","plum3","plum3","plum3","plum3","plum3",
                                                     "lightskyblue3","lightskyblue3", "lightskyblue3", "lightskyblue3", "lightskyblue3",
                                                     "mediumaquamarine", "mediumaquamarine", "mediumaquamarine", "mediumaquamarine", "mediumaquamarine",
                                                     "orange2","orange2","orange2","orange2","orange2"), xlab="Duración", names=c(10, 25, 50, 75,100,
                                                                                                                                 10, 25, 50, 75,100,
                                                                                                                                 10, 25, 50, 75,100,
                                                                                                                                 10, 25, 50, 75,100,
                                                                                                                                 10, 25, 50, 75,100),
        ylim=c(0.28,0.45))
legend(x = "topright", legend = c("10k", "20k", "50k","100k", "1000k"), fill = c("pink3", "plum3", "lightskyblue3", "mediumaquamarine", "orange2"), 
       title = "Partículas")
dev.off()

pdf 
  2

Con ```k=10000```

In [487]:
png("10000.png", width = 900, height = 500)
boxplot(datos_2$porcentaje[1:25], datos_2$porcentaje[26:50], datos_2$porcentaje[51:75], datos_2$porcentaje[76:100], datos_2$porcentaje[101:125],
        datos_2$porcentaje[126:150], datos_2$porcentaje[151:175], datos_2$porcentaje[176:200], datos_2$porcentaje[201:225], datos_2$porcentaje[226:250],
        datos_2$porcentaje[251:275], datos_2$porcentaje[276:300], datos_2$porcentaje[301:325], datos_2$porcentaje[326:350], datos_2$porcentaje[351:375],
        datos_2$porcentaje[376:400], datos_2$porcentaje[401:425], datos_2$porcentaje[426:450], datos_2$porcentaje[451:475], datos_2$porcentaje[476:500], 
        datos_2$porcentaje[501:525], datos_2$porcentaje[526:550], datos_2$porcentaje[551:575], datos_2$porcentaje[576:600], datos_2$porcentaje[600:625],
        ylab="Porcentaje de cúmulos filtrados", col=c("pink3","pink3","pink3","pink3","pink3","plum3","plum3","plum3","plum3","plum3",
                                                     "lightskyblue3","lightskyblue3", "lightskyblue3", "lightskyblue3", "lightskyblue3",
                                                     "mediumaquamarine", "mediumaquamarine", "mediumaquamarine", "mediumaquamarine", "mediumaquamarine",
                                                     "orange2","orange2","orange2","orange2","orange2"), xlab="Duración", names=c(10, 25, 50, 75,100,
                                                                                                                                 10, 25, 50, 75,100,
                                                                                                                                 10, 25, 50, 75,100,
                                                                                                                                 10, 25, 50, 75,100,
                                                                                                                                 10, 25, 50, 75,100),
        ylim=c(0.28,0.45))
legend(x = "topright", legend = c("10k", "20k", "50k","100k", "1000k"), fill = c("pink3", "plum3", "lightskyblue3", "mediumaquamarine", "orange2"), 
       title = "Partículas")
dev.off()

pdf 
  2

In [365]:
cor.test(data$Cumulos,data$porcentaje)


	Pearson's product-moment correlation

data:  data$Cumulos and data$porcentaje
t = 10.28, df = 1873, p-value < 2.2e-16
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.1877928 0.2735044
sample estimates:
     cor 
0.231097 


In [366]:
cor.test(data$Particulas,data$porcentaje)


	Pearson's product-moment correlation

data:  data$Particulas and data$porcentaje
t = 1.3385, df = 1873, p-value = 0.1809
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.01437476  0.07607634
sample estimates:
       cor 
0.03091408 


In [367]:
cor.test(data$t,data$porcentaje)


	Pearson's product-moment correlation

data:  data$t and data$porcentaje
t = -0.099044, df = 1873, p-value = 0.9211
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.04755234  0.04298463
sample estimates:
         cor 
-0.002288547 


## Reto 1:

In [440]:
modelournas1 <- function(k, n, duracion){
    porcentajes <- c()
    originales <- rnorm(k)
    cumulos <- originales - min(originales) + 1
    cumulos <- round(n * cumulos / sum(cumulos))
    #assert(min(cumulos) > 0)
    diferencia <- n - sum(cumulos)
    if (diferencia > 0) {
        for (i in 1:diferencia) {
            p <- sample(1:k, 1)
            cumulos[p] <- cumulos[p] + 1
        }
    } else if (diferencia < 0) {
        for (i in 1:-diferencia) {
            p <- sample(1:k, 1)
            if (cumulos[p] > 1) {
                cumulos[p] <- cumulos[p] - 1
            }
        }
    }
    #assert(length(cumulos[cumulos == 0]) == 0) # que no haya vacios
    #assert(sum(cumulos) == n)
    c <- median(cumulos) # tamanio critico de cumulos
    d <- sd(cumulos) / 4 # factor arbitrario para suavizar la curva
    rotura <- function(x) {
        return (1 / (1 + exp((c - x) / d)))
    }
    union <- function(x) {
        return (exp(-x / c))
    }
    romperse <- function(tam, cuantos) {
        romper <- round(rotura(tam) * cuantos) # independientes
        resultado <- rep(tam, cuantos - romper) # los demas
        if (romper > 0) {
            for (cumulo in 1:romper) { # agregar las rotas
                t <- 1
                if (tam > 2) { # sample no jala con un solo valor
                    t <- sample(1:(tam-1), 1)
                }
                resultado <- c(resultado, t, tam - t)
            }
        }
        #assert(sum(resultado) == tam * cuantos) # no hubo perdidas
        return(resultado)
    }
    unirse <- function(tam, cuantos) {
        unir <- round(union(tam) * cuantos) # independientes
        if (unir > 0) {
            division <- c(rep(-tam, unir), rep(tam, cuantos - unir))
            #assert(sum(abs(division)) == tam * cuantos)
            return(division)
        } else {
            return(rep(tam, cuantos))
        }
    }
    freq <- as.data.frame(table(cumulos))
    names(freq) <- c("tam", "num")
    freq$tam <- as.numeric(levels(freq$tam))[freq$tam]

    for (paso in 1:duracion) {
        #assert(sum(cumulos) == n)
        cumulos <- integer()
        for (i in 1:dim(freq)[1]) { # fase de rotura
            urna <- freq[i,]
            if (urna$tam > 1) { # no tiene caso romper si no se puede
                cumulos <- c(cumulos, romperse(urna$tam, urna$num))
            } else {
                cumulos <- c(cumulos, rep(1, urna$num))
            }
        }
       # assert(sum(cumulos) == n)
       # assert(length(cumulos[cumulos == 0]) == 0) # que no haya vacios
        freq <- as.data.frame(table(cumulos)) # actualizar urnas
        names(freq) <- c("tam", "num")
        freq$tam <- as.numeric(levels(freq$tam))[freq$tam]
       # assert(sum(freq$num * freq$tam) == n)
        cumulos <- integer()
        for (i in 1:dim(freq)[1]) { # fase de union
            urna <- freq[i,]
            cumulos <- c(cumulos, unirse(urna$tam, urna$num))
        }
        #assert(sum(abs(cumulos)) == n)
        #assert(length(cumulos[cumulos == 0]) == 0) # que no haya vacios
        juntarse <- -cumulos[cumulos < 0]
        cumulos <- cumulos[cumulos > 0]
        #assert(sum(cumulos) + sum(juntarse) == n)
        nt <- length(juntarse)
        if (nt > 0) {
            if (nt > 1) {
                juntarse <- sample(juntarse)
                for (i in 1:floor(nt / 2) ) {
                    cumulos <- c(cumulos, juntarse[2*i-1] + juntarse[2*i])
                }
            }
            if (nt %% 2 == 1) {
                cumulos <- c(cumulos, juntarse[nt])
            }
        }
        #assert(sum(cumulos) == n)
        freq <- as.data.frame(table(cumulos))
        names(freq) <- c("tam", "num")
        freq$tam <- as.numeric(levels(freq$tam))[freq$tam]
        #assert(sum(freq$num * freq$tam) == n)
        cum <- c()

        for (i in 1:length(freq$num)){
            if(freq$tam[i]>=c){
                cum <- c(cum, freq$num[i])
            }
        }
        a<-sum(cum)/sum(freq$num)
        porcentajes[paso] <- a 
    }
    
    return(which.max(porcentajes))
}

In [445]:
data_1 <- data.frame()

In [436]:
k=10000
n=1000000
duracion=50

In [446]:
suppressMessages(library(doParallel))
registerDoParallel(makeCluster(detectCores() - 1))
replicas<-30
duracion <-50
for (k in c(100,1000,10000)){
    for (n in c(10*k,100*k,1000*k)){
       resultados <- foreach(i = 1:replicas, .combine=c) %dopar% modelournas1(k,n,duracion)
        for (j in 1:replicas){
            data_1 <- rbind(data_1, c(k,n,duracion, resultados[j]))
        }
    }
}


stopImplicitCluster()

In [506]:
png("reto1.png", width = 900, height = 500)
boxplot(data_1[1:30,4],data_1[31:60,4],data_1[61:90,4], data_1[91:120,4],
        data_1[121:150,4],data_1[151:180,4], data_1[181:210,4], data_1[210:240,4],
        data_1[241:270,4], col=c("pink3", "pink3", "pink3", "lightskyblue3","lightskyblue3", "lightskyblue3", "mediumaquamarine",
                                                  "mediumaquamarine", "mediumaquamarine"), names=c(100,100,100, 1000,1000,1000,10000,10000,10000),
       ylab="Iteración donde alcanza el máximo", xlab="Cantidad de cúmulos")
dev.off()

pdf 
  2

## Reto 2:

In [527]:
names(data_2) <- c("Cúmulos", "Partículas", "Duración", "c", "mayor que c", "menor que c")

In [528]:
data_2

Cúmulos,Partículas,Duración,c,mayor que c,menor que c
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
100,1000,50,1,1,1
100,1000,50,36,1,1
100,1000,50,41,1,1
100,1000,50,26,1,1
100,1000,50,1,1,1
100,1000,50,34,1,1
100,1000,50,29,1,1
100,1000,50,20,1,1
100,1000,50,46,1,1
